In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')
%matplotlib inline

Each of these datasets contains text samples hand-labeled by humans

The data are stored as a CSV and as a pickled pandas dataframe (Python 2.7). Each data file contains 5 columns:

count = number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable by CF).

hate_speech = number of CF users who judged the tweet to be hate speech.

offensive_language = number of CF users who judged the tweet to be offensive.

neither = number of CF users who judged the tweet to be neither offensive nor non-offensive.

class = class label for majority of CF users. 0 - hate speech 1 - offensive language 2 - neither

In [10]:
print(tw_df.shape)
print(wiki_df.shape)

(24783, 7)
(159571, 7)


In [6]:
# 첫번째 모델의 학습을 위한 데이터로는 트위터의 트윗내용에 따른 profanity를 사람이 직접 labeling한 데이터를 사용

# 데이터의 생김새를 살펴보면 다음과 같다.
tw_df = pd.read_csv('data/tw_data.csv')
display(tw_df.head())

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [31]:
tw_texts = tw_df['tweet'].to_frame().rename(columns={'tweet': 'text'})
display(tw_texts)

,text
0,!!! RT @mayasolovely: As a woman you shouldn't...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,"you've gone and broke the wrong heart baby, an..."
24780,young buck wanna eat!!.. dat nigguh like I ain...
24781,youu got wild bitches tellin you lies


In [29]:
tw_labels = tw_df['class'].map(lambda x : 0 if x == 2 else 1).to_frame().rename(columns={'class':'profanity'})
display(tw_labels)

,profanity
0,0
1,1
2,1
3,1
4,1
...,...
24778,1
24779,0
24780,1
24781,1


In [30]:
display(tw_labels['profanity'].value_counts())

1    20620
0     4163
Name: profanity, dtype: int64

In [7]:
# 두번째 데이터로는 위키피디아의 악성글을 사람이 직접 labeling한 데이터를 사용
# 데이터의 상세는 다음과 같다.

wiki_df = pd.read_csv('data/wiki_train.csv').drop(columns='id')
display(wiki_df.head())

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [25]:
wiki_text = wiki_df['comment_text'].to_frame().rename(columns={'comment_text':'text'})
display(wiki_text)

,text
0,Explanation\nWhy the edits made under my usern...
1,D'aww! He matches this background colour I'm s...
2,"Hey man, I'm really not trying to edit war. It..."
3,"""\nMore\nI can't make any real suggestions on ..."
4,"You, sir, are my hero. Any chance you remember..."
...,...
159566,""":::::And for the second time of asking, when ..."
159567,You should be ashamed of yourself \n\nThat is ...
159568,"Spitzer \n\nUmm, theres no actual article for ..."
159569,And it looks like it was actually you who put ...


In [18]:
wiki_labels = wiki_df.iloc[:, 1:].sum(axis=1).map(lambda x : 1 if x > 0 else 0).to_frame().rename(columns={0:'profanity'})
display(wiki_labels['profanity'].value_counts())

0    143346
1     16225
Name: profanity, dtype: int64

In [ ]:
from ml_model.d_pre import tokenize, make_features, make_features2

